<a href="https://colab.research.google.com/github/rogerfvieira/fare_prediction/blob/main/fare_prediction(linear_regression).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install --upgrade google-cloud-bigquery
# !pip install google-colab

# Fare prediction (Linear_regression) using bigqeury ML

## Imports


In [1]:
from google.cloud import bigquery 
from google.colab import auth
import pandas as pd

In [2]:
auth.authenticate_user()

## Cursor creation

In [3]:
project_id = 'play-368717'
dataset_id = 'linear_regression'
client = bigquery.Client(project=project_id)

## Unclean data source

In [4]:
unclean_data_query = """
SELECT *
FROM bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022
LIMIT 100
"""

df_unclean = client.query(unclean_data_query).to_dataframe()

In [5]:
df_unclean

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code,store_and_fwd_flag,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,imp_surcharge,airport_fee,total_amount,pickup_location_id,dropoff_location_id,data_file_year,data_file_month
0,2,2022-02-18 21:03:05+00:00,2022-02-18 21:03:14+00:00,1,0E-9,1.0,N,1,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,193,193,2022,2
1,1,2022-02-16 12:38:32+00:00,2022-02-16 12:39:38+00:00,1,0.800000000,5.0,N,3,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,132,132,2022,2
2,2,2022-02-17 17:32:09+00:00,2022-02-17 17:32:30+00:00,1,0E-9,1.0,N,1,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,7,7,2022,2
3,1,2022-02-24 15:19:48+00:00,2022-02-24 15:25:40+00:00,1,0.800000000,1.0,N,4,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,68,90,2022,2
4,2,2022-02-09 13:58:27+00:00,2022-02-09 13:59:13+00:00,1,0E-9,1.0,N,1,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,193,193,2022,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,2022-02-25 17:03:58+00:00,2022-02-25 17:03:58+00:00,0,0E-9,1.0,N,2,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,234,264,2022,2
96,1,2022-02-22 16:24:15+00:00,2022-02-22 16:25:22+00:00,1,0E-9,1.0,N,1,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,145,145,2022,2
97,1,2022-02-22 20:51:01+00:00,2022-02-22 20:55:40+00:00,1,0.900000000,1.0,N,4,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,246,90,2022,2
98,2,2022-02-25 20:24:58+00:00,2022-02-25 20:25:20+00:00,1,0E-9,1.0,N,1,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,0E-9,193,193,2022,2


## Data Cleaning / feature engineering and selection

In [6]:
training_data_query = """
     CREATE OR REPLACE TABLE `play-368717.linear_regression.training_data_fare_prediction` AS(
  SELECT vendor_id,
         pickup_datetime,
         dropoff_datetime,
         passenger_count,
         trip_distance,
         pickup_location_id ,
         dropoff_location_id,
         fare_amount
  FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2022`
  WHERE pickup_datetime IS NOT NULL
  AND dropoff_datetime IS NOT NULL
  AND passenger_count IS NOT NULL
  AND trip_distance IS NOT NULL AND trip_distance!=0
  AND pickup_location_id IS NOT NULL
  AND dropoff_location_id IS NOT NULL
  AND fare_amount IS NOT NULL
  )
"""
training_data= client.query(training_data_query)

## Clean data

In [7]:
dataset_ref = bigquery.DatasetReference(project_id, dataset_id)
table_ref = dataset_ref.table("training_data_fare_prediction")
table = client.get_table(table_ref)
df_training_data = client.list_rows(table).to_dataframe()

In [8]:
df_training_data

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_location_id,dropoff_location_id,fare_amount
0,2,2022-02-16 12:51:18+00:00,2022-02-16 13:08:46+00:00,1,2.770000000,89,149,14.000000000
1,1,2022-02-02 18:59:49+00:00,2022-02-02 19:26:17+00:00,1,6.500000000,70,160,23.000000000
2,2,2022-02-01 15:13:26+00:00,2022-02-01 15:19:53+00:00,1,0.770000000,179,7,5.500000000
3,2,2022-02-12 04:51:57+00:00,2022-02-12 04:56:34+00:00,1,1.010000000,193,145,5.500000000
4,2,2022-02-23 13:00:22+00:00,2022-02-23 13:36:57+00:00,1,11.330000000,70,181,36.000000000
...,...,...,...,...,...,...,...,...
27982186,1,2022-09-22 00:49:21+00:00,2022-09-22 01:11:07+00:00,2,9.800000000,264,264,29.500000000
27982187,2,2022-01-08 19:46:00+00:00,2022-01-08 19:54:15+00:00,2,1.210000000,264,264,7.000000000
27982188,1,2022-01-21 11:15:14+00:00,2022-01-21 11:23:53+00:00,1,1.100000000,264,264,7.000000000
27982189,2,2022-01-25 20:48:33+00:00,2022-01-25 20:55:50+00:00,5,1.810000000,264,264,7.000000000


## Model Creation

In [13]:
model_creation_query = """
CREATE OR REPLACE MODEL `play-368717.linear_regression.fare_prediction_model` OPTIONS(MODEL_TYPE='LINEAR_REG',LABELS=['fare_amount']) AS
SELECT * FROM `play-368717.linear_regression.training_data_fare_prediction`;
"""

In [14]:
fare_prediction_model = client.query(model_creation_query)

In [16]:
models = client.list_models('linear_regression')

## Model Evaluation

In [17]:
model_evaluation_query = '''
SELECT * FROM ML.EVALUATE(MODEL `play-368717.linear_regression.fare_prediction_model`)
'''

In [20]:
model_eval =  client.query(model_evaluation_query).to_dataframe()

In [21]:
model_eval

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,4.357807,54.032621,0.151982,2.967457,0.684807,0.684807
